### Criando a conexão

In [3]:
import psycopg2 #biblioteca para conectar ao banco
import pandas as pd #biblioteca para criação e gerenciamento de data frame. Também utilizada para exportar o dataframe para csv
import pandas.io.sql as sqlio #utilizado para integrar o sql com pandas, permitindo exportar os dados direto do banco para um dataframe

In [4]:
conn = psycopg2.connect(    
    database="conectcar",
    user="raul_correa", 
    password="Conect@091!",
    host="sdlf-cntcar-redshift-dev.capkkn406tig.us-east-2.redshift.amazonaws.com",
    port='5439'
)

In [5]:
#version 1.0 - DESCONTINUADO
sql = "WITH  cte_MesRef AS (      SELECT                              to_char(date_trunc('month', current_date - interval '7 month'), 'YYYYMM') as MesRefVar ),                   cte_baseHabilitada as (       	select distinct      	d.documento*1 :: float as documento,      	date_part(year, a.DataHabilitada) ||lpad(date_part(month, a.DataHabilitada),2,'0') as anomes,      	a.agregado as agregado,      	date_part(year, b.datacancelamento) ||lpad(date_part(month, b.datacancelamento),2,'0') as anoMesCancelamento      	from cntcar_work.vw_redshift_ciclodevida_basehabilitada_stage as a      	left join cntcar_work.tb_conectcar_dbo_adesao_stage as b on a.adesaoid = b.adesaoid                        left join cntcar_work.tb_conectcar_dbo_veiculo_stage as c on b.veiculoID = c.veiculo_id      	left join conectcar.cntcar_work.tb_conectcar_dbo_cliente_stage as d on b.Clienteid = d.Clienteid      	      	where anomes = (SELECT MesRefVar FROM cte_MesRef)       	and len(documento) <= 11       ),      cte_flagPlacasas as (      	select       	documento,      	count (documento) as flagPlacas      	from cte_baseHabilitada      	group by documento       	having flagPlacas > 1      	order by flagPlacas desc      ),      cte_BaseHabilitadaPFCom1PlacaSemPorto as (      select       	a.documento ::float,      	a.anomes ::int,      	a.agregado,      	a.anomescancelamento::int       from cte_baseHabilitada a      left join cte_flagPlacasas b      on a.documento = b.documento      where b.flagPlacas is NULL      ),      cte_baseHabilitadaPorto as (      	select distinct      	regexp_replace(c.codigointernoparceiro,'[^0-9]+', '') as Documento,      	date_part(year, bh.datahabilitada) ||lpad(date_part(month, bh.datahabilitada),2,'0') as anomes,      	p.agregado,      	date_part(year, a.datacancelamento) ||lpad(date_part(month, a.datacancelamento),2,'0') as anomescancelamento       from      	cntcar_work.vw_redshift_ciclodevida_basehabilitada_stage bh      left outer join cntcar_work.tb_parceiro_dbo_ativacaocontrato_stage ac on ac.adesaoid = bh.adesaoid      left outer join cntcar_work.tb_parceiro_dbo_contrato_stage c on c.contratoid = ac.contratoid      left outer join      cntcar_work.vw_redshift_ciclodevida_ativacao a on a.adesaoid = ac.adesaoid       left outer join      cntcar_work.vw_ciclodevida_portifolio p on p.adesaoid = bh.adesaoid      where       	upper(p.agregado) = 'PORTO SEGURO'      	and upper(p.detalhado) not in ('CARROFACIL','CARRO FACIL','CARRO FÁCIL')      	and upper(a.nomeplano) <> 'LOCADORA'       	and lpad(date_part(year, cast(bh.datahabilitada as date)), 4, '0') ||lpad(date_part(month, cast(bh.datahabilitada as date)),2,'0') = (SELECT MesRefVar FROM cte_MesRef)       order by       	1, 2      ),      cte_flagPlacaSPorto as (      	select       	documento,      	count (documento) as flagPlacas      	from cte_baseHabilitadaPorto      	group by documento       	having flagPlacas > 1      	order by flagPlacas desc      ),      cte_BaseHabilitadaPFCom1Porto as (      	select      	a.documento::float,      	a.anomes ::int,      	a.agregado,      	a.anomescancelamento::int      	from cte_baseHabilitadaPorto a      	left join cte_flagPlacaSPorto b on a.documento = b.documento      	where b.flagPlacas is null and len(a.documento) <=11      ),      cte_BasePF_Clientes1Placa as (      	select * from cte_BaseHabilitadaPFCom1PlacaSemPorto      union      	select * from cte_BaseHabilitadaPFCom1Porto      )      select * from cte_BasePF_Clientes1Placa       "

### Puxando todos os meses de bases habilitadas

In [26]:
#version 2.0
cte_BasePF_Clientes1Placa_09 = "WITH cte_MesRef_09 AS (  select to_char(date_trunc('month', current_date - interval '9 month'), 'YYYYMM') as MesRefVar  ),    cte_baseHabilitada_09 as (  select distinct  d.documento*1 :: float as documento,  date_part(year, a.DataHabilitada) ||lpad(date_part(month, a.DataHabilitada),2,'0') as anomes,  a.agregado as agregado,  date_part(year, b.datacancelamento) ||lpad(date_part(month, b.datacancelamento),2,'0') as anoMesCancelamento  from cntcar_work.vw_redshift_ciclodevida_basehabilitada_stage as a  left join cntcar_work.tb_conectcar_dbo_adesao_stage as b on a.adesaoid = b.adesaoid  left join cntcar_work.tb_conectcar_dbo_veiculo_stage as c on b.veiculoID = c.veiculo_id  left join conectcar.cntcar_work.tb_conectcar_dbo_cliente_stage as d on b.Clienteid = d.Clienteid  where anomes = (SELECT MesRefVar FROM cte_MesRef_09)  and len(documento) <= 11  ),    cte_flagPlacasas_09 as (  select  documento,  count (documento) as flagPlacas  from cte_baseHabilitada_09  group by documento  having flagPlacas > 1  order by flagPlacas desc  ),    cte_baseHabilitada_09PFCom1PlacaSemPorto as (  select  a.documento ::float,  a.anomes ::int,  a.agregado,  a.anomescancelamento::int  from cte_baseHabilitada_09 a  left join cte_flagPlacasas_09 b  on a.documento = b.documento  where b.flagPlacas is NULL  ),    cte_baseHabilitada_09Porto as (  select distinct  regexp_replace(c.codigointernoparceiro,'[^0-9]+', '') as Documento,  date_part(year, bh.datahabilitada) ||lpad(date_part(month, bh.datahabilitada),2,'0') as anomes,  p.agregado,  date_part(year, a.datacancelamento) ||lpad(date_part(month, a.datacancelamento),2,'0') as anomescancelamento  from cntcar_work.vw_redshift_ciclodevida_basehabilitada_stage bh  left outer join cntcar_work.tb_parceiro_dbo_ativacaocontrato_stage ac on ac.adesaoid = bh.adesaoid  left outer join cntcar_work.tb_parceiro_dbo_contrato_stage c on c.contratoid = ac.contratoid  left outer join cntcar_work.vw_redshift_ciclodevida_ativacao a on a.adesaoid = ac.adesaoid  left outer join cntcar_work.vw_ciclodevida_portifolio p on p.adesaoid = bh.adesaoid  where  upper(p.agregado) = 'PORTO SEGURO'  and upper(p.detalhado) not in ('CARROFACIL','CARRO FACIL','CARRO FÁCIL')  and upper(a.nomeplano) <> 'LOCADORA'  and lpad(date_part(year, cast(bh.datahabilitada as date)), 4, '0') ||lpad(date_part(month, cast(bh.datahabilitada as date)),2,'0') = (SELECT MesRefVar FROM cte_MesRef_09)  order by  1, 2  ),    cte_flagPlacasasPorto_09 as (  select  documento,  count (documento) as flagPlacas  from cte_baseHabilitada_09Porto  group by documento  having flagPlacas > 1  order by flagPlacas desc  ),    cte_baseHabilitada_09PFCom1Porto as (  select  a.documento::float,  a.anomes ::int,  a.agregado,  a.anomescancelamento::int  from cte_baseHabilitada_09Porto a  left join cte_flagPlacasasPorto_09 b on a.documento = b.documento  where b.flagPlacas is null and len(a.documento) <=11  ),    cte_BasePF_Clientes1Placa_09 as (  select * from cte_baseHabilitada_09PFCom1PlacaSemPorto  union  select * from cte_baseHabilitada_09PFCom1Porto  )    select * from cte_BasePF_Clientes1Placa_09 "

In [25]:
cte_BasePF_Clientes1Placa_08 = "WITH cte_MesRef_08 AS (  select to_char(date_trunc('month', current_date - interval '8 month'), 'YYYYMM') as MesRefVar  ),    cte_baseHabilitada_08 as (  select distinct  d.documento*1 :: float as documento,  date_part(year, a.DataHabilitada) ||lpad(date_part(month, a.DataHabilitada),2,'0') as anomes,  a.agregado as agregado,  date_part(year, b.datacancelamento) ||lpad(date_part(month, b.datacancelamento),2,'0') as anoMesCancelamento  from cntcar_work.vw_redshift_ciclodevida_basehabilitada_stage as a  left join cntcar_work.tb_conectcar_dbo_adesao_stage as b on a.adesaoid = b.adesaoid  left join cntcar_work.tb_conectcar_dbo_veiculo_stage as c on b.veiculoID = c.veiculo_id  left join conectcar.cntcar_work.tb_conectcar_dbo_cliente_stage as d on b.Clienteid = d.Clienteid  where anomes = (SELECT MesRefVar FROM cte_MesRef_08)  and len(documento) <= 11  ),    cte_flagPlacasas_08 as (  select  documento,  count (documento) as flagPlacas  from cte_baseHabilitada_08  group by documento  having flagPlacas > 1  order by flagPlacas desc  ),    cte_baseHabilitada_08PFCom1PlacaSemPorto as (  select  a.documento ::float,  a.anomes ::int,  a.agregado,  a.anomescancelamento::int  from cte_baseHabilitada_08 a  left join cte_flagPlacasas_08 b  on a.documento = b.documento  where b.flagPlacas is NULL  ),    cte_baseHabilitada_08Porto as (  select distinct  regexp_replace(c.codigointernoparceiro,'[^0-9]+', '') as Documento,  date_part(year, bh.datahabilitada) ||lpad(date_part(month, bh.datahabilitada),2,'0') as anomes,  p.agregado,  date_part(year, a.datacancelamento) ||lpad(date_part(month, a.datacancelamento),2,'0') as anomescancelamento  from cntcar_work.vw_redshift_ciclodevida_basehabilitada_stage bh  left outer join cntcar_work.tb_parceiro_dbo_ativacaocontrato_stage ac on ac.adesaoid = bh.adesaoid  left outer join cntcar_work.tb_parceiro_dbo_contrato_stage c on c.contratoid = ac.contratoid  left outer join cntcar_work.vw_redshift_ciclodevida_ativacao a on a.adesaoid = ac.adesaoid  left outer join cntcar_work.vw_ciclodevida_portifolio p on p.adesaoid = bh.adesaoid  where  upper(p.agregado) = 'PORTO SEGURO'  and upper(p.detalhado) not in ('CARROFACIL','CARRO FACIL','CARRO FÁCIL')  and upper(a.nomeplano) <> 'LOCADORA'  and lpad(date_part(year, cast(bh.datahabilitada as date)), 4, '0') ||lpad(date_part(month, cast(bh.datahabilitada as date)),2,'0') = (SELECT MesRefVar FROM cte_MesRef_08)  order by  1, 2  ),    cte_flagPlacasasPorto_08 as (  select  documento,  count (documento) as flagPlacas  from cte_baseHabilitada_08Porto  group by documento  having flagPlacas > 1  order by flagPlacas desc  ),    cte_baseHabilitada_08PFCom1Porto as (  select  a.documento::float,  a.anomes ::int,  a.agregado,  a.anomescancelamento::int  from cte_baseHabilitada_08Porto a  left join cte_flagPlacasasPorto_08 b on a.documento = b.documento  where b.flagPlacas is null and len(a.documento) <=11  ),    cte_BasePF_Clientes1Placa_08 as (  select * from cte_baseHabilitada_08PFCom1PlacaSemPorto  union  select * from cte_baseHabilitada_08PFCom1Porto  )    select * from cte_BasePF_Clientes1Placa_08 "

In [24]:
cte_BasePF_Clientes1Placa_07 = "WITH cte_MesRef_07 AS (  select to_char(date_trunc('month', current_date - interval '7 month'), 'YYYYMM') as MesRefVar  ),    cte_baseHabilitada_07 as (  select distinct  d.documento*1 :: float as documento,  date_part(year, a.DataHabilitada) ||lpad(date_part(month, a.DataHabilitada),2,'0') as anomes,  a.agregado as agregado,  date_part(year, b.datacancelamento) ||lpad(date_part(month, b.datacancelamento),2,'0') as anoMesCancelamento  from cntcar_work.vw_redshift_ciclodevida_basehabilitada_stage as a  left join cntcar_work.tb_conectcar_dbo_adesao_stage as b on a.adesaoid = b.adesaoid  left join cntcar_work.tb_conectcar_dbo_veiculo_stage as c on b.veiculoID = c.veiculo_id  left join conectcar.cntcar_work.tb_conectcar_dbo_cliente_stage as d on b.Clienteid = d.Clienteid  where anomes = (SELECT MesRefVar FROM cte_MesRef_07)  and len(documento) <= 11  ),    cte_flagPlacasas_07 as (  select  documento,  count (documento) as flagPlacas  from cte_baseHabilitada_07  group by documento  having flagPlacas > 1  order by flagPlacas desc  ),    cte_baseHabilitada_07PFCom1PlacaSemPorto as (  select  a.documento ::float,  a.anomes ::int,  a.agregado,  a.anomescancelamento::int  from cte_baseHabilitada_07 a  left join cte_flagPlacasas_07 b  on a.documento = b.documento  where b.flagPlacas is NULL  ),    cte_baseHabilitada_07Porto as (  select distinct  regexp_replace(c.codigointernoparceiro,'[^0-9]+', '') as Documento,  date_part(year, bh.datahabilitada) ||lpad(date_part(month, bh.datahabilitada),2,'0') as anomes,  p.agregado,  date_part(year, a.datacancelamento) ||lpad(date_part(month, a.datacancelamento),2,'0') as anomescancelamento  from cntcar_work.vw_redshift_ciclodevida_basehabilitada_stage bh  left outer join cntcar_work.tb_parceiro_dbo_ativacaocontrato_stage ac on ac.adesaoid = bh.adesaoid  left outer join cntcar_work.tb_parceiro_dbo_contrato_stage c on c.contratoid = ac.contratoid  left outer join cntcar_work.vw_redshift_ciclodevida_ativacao a on a.adesaoid = ac.adesaoid  left outer join cntcar_work.vw_ciclodevida_portifolio p on p.adesaoid = bh.adesaoid  where  upper(p.agregado) = 'PORTO SEGURO'  and upper(p.detalhado) not in ('CARROFACIL','CARRO FACIL','CARRO FÁCIL')  and upper(a.nomeplano) <> 'LOCADORA'  and lpad(date_part(year, cast(bh.datahabilitada as date)), 4, '0') ||lpad(date_part(month, cast(bh.datahabilitada as date)),2,'0') = (SELECT MesRefVar FROM cte_MesRef_07)  order by  1, 2  ),    cte_flagPlacasasPorto_07 as (  select  documento,  count (documento) as flagPlacas  from cte_baseHabilitada_07Porto  group by documento  having flagPlacas > 1  order by flagPlacas desc  ),    cte_baseHabilitada_07PFCom1Porto as (  select  a.documento::float,  a.anomes ::int,  a.agregado,  a.anomescancelamento::int  from cte_baseHabilitada_07Porto a  left join cte_flagPlacasasPorto_07 b on a.documento = b.documento  where b.flagPlacas is null and len(a.documento) <=11  ),    cte_BasePF_Clientes1Placa_07 as (  select * from cte_baseHabilitada_07PFCom1PlacaSemPorto  union  select * from cte_baseHabilitada_07PFCom1Porto  )    select * from cte_BasePF_Clientes1Placa_07 "

In [23]:
cte_BasePF_Clientes1Placa_06 = "WITH cte_MesRef_06 AS (  select to_char(date_trunc('month', current_date - interval '6 month'), 'YYYYMM') as MesRefVar  ),    cte_baseHabilitada_06 as (  select distinct  d.documento*1 :: float as documento,  date_part(year, a.DataHabilitada) ||lpad(date_part(month, a.DataHabilitada),2,'0') as anomes,  a.agregado as agregado,  date_part(year, b.datacancelamento) ||lpad(date_part(month, b.datacancelamento),2,'0') as anoMesCancelamento  from cntcar_work.vw_redshift_ciclodevida_basehabilitada_stage as a  left join cntcar_work.tb_conectcar_dbo_adesao_stage as b on a.adesaoid = b.adesaoid  left join cntcar_work.tb_conectcar_dbo_veiculo_stage as c on b.veiculoID = c.veiculo_id  left join conectcar.cntcar_work.tb_conectcar_dbo_cliente_stage as d on b.Clienteid = d.Clienteid  where anomes = (SELECT MesRefVar FROM cte_MesRef_06)  and len(documento) <= 11  ),    cte_flagPlacasas_06 as (  select  documento,  count (documento) as flagPlacas  from cte_baseHabilitada_06  group by documento  having flagPlacas > 1  order by flagPlacas desc  ),    cte_baseHabilitada_06PFCom1PlacaSemPorto as (  select  a.documento ::float,  a.anomes ::int,  a.agregado,  a.anomescancelamento::int  from cte_baseHabilitada_06 a  left join cte_flagPlacasas_06 b  on a.documento = b.documento  where b.flagPlacas is NULL  ),    cte_baseHabilitada_06Porto as (  select distinct  regexp_replace(c.codigointernoparceiro,'[^0-9]+', '') as Documento,  date_part(year, bh.datahabilitada) ||lpad(date_part(month, bh.datahabilitada),2,'0') as anomes,  p.agregado,  date_part(year, a.datacancelamento) ||lpad(date_part(month, a.datacancelamento),2,'0') as anomescancelamento  from cntcar_work.vw_redshift_ciclodevida_basehabilitada_stage bh  left outer join cntcar_work.tb_parceiro_dbo_ativacaocontrato_stage ac on ac.adesaoid = bh.adesaoid  left outer join cntcar_work.tb_parceiro_dbo_contrato_stage c on c.contratoid = ac.contratoid  left outer join cntcar_work.vw_redshift_ciclodevida_ativacao a on a.adesaoid = ac.adesaoid  left outer join cntcar_work.vw_ciclodevida_portifolio p on p.adesaoid = bh.adesaoid  where  upper(p.agregado) = 'PORTO SEGURO'  and upper(p.detalhado) not in ('CARROFACIL','CARRO FACIL','CARRO FÁCIL')  and upper(a.nomeplano) <> 'LOCADORA'  and lpad(date_part(year, cast(bh.datahabilitada as date)), 4, '0') ||lpad(date_part(month, cast(bh.datahabilitada as date)),2,'0') = (SELECT MesRefVar FROM cte_MesRef_06)  order by  1, 2  ),    cte_flagPlacasasPorto_06 as (  select  documento,  count (documento) as flagPlacas  from cte_baseHabilitada_06Porto  group by documento  having flagPlacas > 1  order by flagPlacas desc  ),    cte_baseHabilitada_06PFCom1Porto as (  select  a.documento::float,  a.anomes ::int,  a.agregado,  a.anomescancelamento::int  from cte_baseHabilitada_06Porto a  left join cte_flagPlacasasPorto_06 b on a.documento = b.documento  where b.flagPlacas is null and len(a.documento) <=11  ),    cte_BasePF_Clientes1Placa_06 as (  select * from cte_baseHabilitada_06PFCom1PlacaSemPorto  union  select * from cte_baseHabilitada_06PFCom1Porto  )    select * from cte_BasePF_Clientes1Placa_06 "

In [22]:
cte_BasePF_Clientes1Placa_05 = "WITH cte_MesRef_05 AS (  select to_char(date_trunc('month', current_date - interval '5 month'), 'YYYYMM') as MesRefVar  ),    cte_baseHabilitada_05 as (  select distinct  d.documento*1 :: float as documento,  date_part(year, a.DataHabilitada) ||lpad(date_part(month, a.DataHabilitada),2,'0') as anomes,  a.agregado as agregado,  date_part(year, b.datacancelamento) ||lpad(date_part(month, b.datacancelamento),2,'0') as anoMesCancelamento  from cntcar_work.vw_redshift_ciclodevida_basehabilitada_stage as a  left join cntcar_work.tb_conectcar_dbo_adesao_stage as b on a.adesaoid = b.adesaoid  left join cntcar_work.tb_conectcar_dbo_veiculo_stage as c on b.veiculoID = c.veiculo_id  left join conectcar.cntcar_work.tb_conectcar_dbo_cliente_stage as d on b.Clienteid = d.Clienteid  where anomes = (SELECT MesRefVar FROM cte_MesRef_05)  and len(documento) <= 11  ),    cte_flagPlacasas_05 as (  select  documento,  count (documento) as flagPlacas  from cte_baseHabilitada_05  group by documento  having flagPlacas > 1  order by flagPlacas desc  ),    cte_baseHabilitada_05PFCom1PlacaSemPorto as (  select  a.documento ::float,  a.anomes ::int,  a.agregado,  a.anomescancelamento::int  from cte_baseHabilitada_05 a  left join cte_flagPlacasas_05 b  on a.documento = b.documento  where b.flagPlacas is NULL  ),    cte_baseHabilitada_05Porto as (  select distinct  regexp_replace(c.codigointernoparceiro,'[^0-9]+', '') as Documento,  date_part(year, bh.datahabilitada) ||lpad(date_part(month, bh.datahabilitada),2,'0') as anomes,  p.agregado,  date_part(year, a.datacancelamento) ||lpad(date_part(month, a.datacancelamento),2,'0') as anomescancelamento  from cntcar_work.vw_redshift_ciclodevida_basehabilitada_stage bh  left outer join cntcar_work.tb_parceiro_dbo_ativacaocontrato_stage ac on ac.adesaoid = bh.adesaoid  left outer join cntcar_work.tb_parceiro_dbo_contrato_stage c on c.contratoid = ac.contratoid  left outer join cntcar_work.vw_redshift_ciclodevida_ativacao a on a.adesaoid = ac.adesaoid  left outer join cntcar_work.vw_ciclodevida_portifolio p on p.adesaoid = bh.adesaoid  where  upper(p.agregado) = 'PORTO SEGURO'  and upper(p.detalhado) not in ('CARROFACIL','CARRO FACIL','CARRO FÁCIL')  and upper(a.nomeplano) <> 'LOCADORA'  and lpad(date_part(year, cast(bh.datahabilitada as date)), 4, '0') ||lpad(date_part(month, cast(bh.datahabilitada as date)),2,'0') = (SELECT MesRefVar FROM cte_MesRef_05)  order by  1, 2  ),    cte_flagPlacasasPorto_05 as (  select  documento,  count (documento) as flagPlacas  from cte_baseHabilitada_05Porto  group by documento  having flagPlacas > 1  order by flagPlacas desc  ),    cte_baseHabilitada_05PFCom1Porto as (  select  a.documento::float,  a.anomes ::int,  a.agregado,  a.anomescancelamento::int  from cte_baseHabilitada_05Porto a  left join cte_flagPlacasasPorto_05 b on a.documento = b.documento  where b.flagPlacas is null and len(a.documento) <=11  ),    cte_BasePF_Clientes1Placa_05 as (  select * from cte_baseHabilitada_05PFCom1PlacaSemPorto  union  select * from cte_baseHabilitada_05PFCom1Porto  )    select * from cte_BasePF_Clientes1Placa_05 "

In [27]:
cte_BasePF_Clientes1Placa_04 = "WITH cte_MesRef_04 AS (  select to_char(date_trunc('month', current_date - interval '4 month'), 'YYYYMM') as MesRefVar  ),    cte_baseHabilitada_04 as (  select distinct  d.documento*1 :: float as documento,  date_part(year, a.DataHabilitada) ||lpad(date_part(month, a.DataHabilitada),2,'0') as anomes,  a.agregado as agregado,  date_part(year, b.datacancelamento) ||lpad(date_part(month, b.datacancelamento),2,'0') as anoMesCancelamento  from cntcar_work.vw_redshift_ciclodevida_basehabilitada_stage as a  left join cntcar_work.tb_conectcar_dbo_adesao_stage as b on a.adesaoid = b.adesaoid  left join cntcar_work.tb_conectcar_dbo_veiculo_stage as c on b.veiculoID = c.veiculo_id  left join conectcar.cntcar_work.tb_conectcar_dbo_cliente_stage as d on b.Clienteid = d.Clienteid  where anomes = (SELECT MesRefVar FROM cte_MesRef_04)  and len(documento) <= 11  ),    cte_flagPlacasas_04 as (  select  documento,  count (documento) as flagPlacas  from cte_baseHabilitada_04  group by documento  having flagPlacas > 1  order by flagPlacas desc  ),    cte_baseHabilitada_04PFCom1PlacaSemPorto as (  select  a.documento ::float,  a.anomes ::int,  a.agregado,  a.anomescancelamento::int  from cte_baseHabilitada_04 a  left join cte_flagPlacasas_04 b  on a.documento = b.documento  where b.flagPlacas is NULL  ),    cte_baseHabilitada_04Porto as (  select distinct  regexp_replace(c.codigointernoparceiro,'[^0-9]+', '') as Documento,  date_part(year, bh.datahabilitada) ||lpad(date_part(month, bh.datahabilitada),2,'0') as anomes,  p.agregado,  date_part(year, a.datacancelamento) ||lpad(date_part(month, a.datacancelamento),2,'0') as anomescancelamento  from cntcar_work.vw_redshift_ciclodevida_basehabilitada_stage bh  left outer join cntcar_work.tb_parceiro_dbo_ativacaocontrato_stage ac on ac.adesaoid = bh.adesaoid  left outer join cntcar_work.tb_parceiro_dbo_contrato_stage c on c.contratoid = ac.contratoid  left outer join cntcar_work.vw_redshift_ciclodevida_ativacao a on a.adesaoid = ac.adesaoid  left outer join cntcar_work.vw_ciclodevida_portifolio p on p.adesaoid = bh.adesaoid  where  upper(p.agregado) = 'PORTO SEGURO'  and upper(p.detalhado) not in ('CARROFACIL','CARRO FACIL','CARRO FÁCIL')  and upper(a.nomeplano) <> 'LOCADORA'  and lpad(date_part(year, cast(bh.datahabilitada as date)), 4, '0') ||lpad(date_part(month, cast(bh.datahabilitada as date)),2,'0') = (SELECT MesRefVar FROM cte_MesRef_04)  order by  1, 2  ),    cte_flagPlacasasPorto_04 as (  select  documento,  count (documento) as flagPlacas  from cte_baseHabilitada_04Porto  group by documento  having flagPlacas > 1  order by flagPlacas desc  ),    cte_baseHabilitada_04PFCom1Porto as (  select  a.documento::float,  a.anomes ::int,  a.agregado,  a.anomescancelamento::int  from cte_baseHabilitada_04Porto a  left join cte_flagPlacasasPorto_04 b on a.documento = b.documento  where b.flagPlacas is null and len(a.documento) <=11  ),    cte_BasePF_Clientes1Placa_04 as (  select * from cte_baseHabilitada_04PFCom1PlacaSemPorto  union  select * from cte_baseHabilitada_04PFCom1Porto  )    select * from cte_BasePF_Clientes1Placa_04 "

In [29]:
cte_BasePF_Clientes1Placa_03 = "WITH cte_MesRef_03 AS (  select to_char(date_trunc('month', current_date - interval '3 month'), 'YYYYMM') as MesRefVar  ),    cte_baseHabilitada_03 as (  select distinct  d.documento*1 :: float as documento,  date_part(year, a.DataHabilitada) ||lpad(date_part(month, a.DataHabilitada),2,'0') as anomes,  a.agregado as agregado,  date_part(year, b.datacancelamento) ||lpad(date_part(month, b.datacancelamento),2,'0') as anoMesCancelamento  from cntcar_work.vw_redshift_ciclodevida_basehabilitada_stage as a  left join cntcar_work.tb_conectcar_dbo_adesao_stage as b on a.adesaoid = b.adesaoid  left join cntcar_work.tb_conectcar_dbo_veiculo_stage as c on b.veiculoID = c.veiculo_id  left join conectcar.cntcar_work.tb_conectcar_dbo_cliente_stage as d on b.Clienteid = d.Clienteid  where anomes = (SELECT MesRefVar FROM cte_MesRef_03)  and len(documento) <= 11  ),    cte_flagPlacasas_03 as (  select  documento,  count (documento) as flagPlacas  from cte_baseHabilitada_03  group by documento  having flagPlacas > 1  order by flagPlacas desc  ),    cte_baseHabilitada_03PFCom1PlacaSemPorto as (  select  a.documento ::float,  a.anomes ::int,  a.agregado,  a.anomescancelamento::int  from cte_baseHabilitada_03 a  left join cte_flagPlacasas_03 b  on a.documento = b.documento  where b.flagPlacas is NULL  ),    cte_baseHabilitada_03Porto as (  select distinct  regexp_replace(c.codigointernoparceiro,'[^0-9]+', '') as Documento,  date_part(year, bh.datahabilitada) ||lpad(date_part(month, bh.datahabilitada),2,'0') as anomes,  p.agregado,  date_part(year, a.datacancelamento) ||lpad(date_part(month, a.datacancelamento),2,'0') as anomescancelamento  from cntcar_work.vw_redshift_ciclodevida_basehabilitada_stage bh  left outer join cntcar_work.tb_parceiro_dbo_ativacaocontrato_stage ac on ac.adesaoid = bh.adesaoid  left outer join cntcar_work.tb_parceiro_dbo_contrato_stage c on c.contratoid = ac.contratoid  left outer join cntcar_work.vw_redshift_ciclodevida_ativacao a on a.adesaoid = ac.adesaoid  left outer join cntcar_work.vw_ciclodevida_portifolio p on p.adesaoid = bh.adesaoid  where  upper(p.agregado) = 'PORTO SEGURO'  and upper(p.detalhado) not in ('CARROFACIL','CARRO FACIL','CARRO FÁCIL')  and upper(a.nomeplano) <> 'LOCADORA'  and lpad(date_part(year, cast(bh.datahabilitada as date)), 4, '0') ||lpad(date_part(month, cast(bh.datahabilitada as date)),2,'0') = (SELECT MesRefVar FROM cte_MesRef_03)  order by  1, 2  ),    cte_flagPlacasasPorto_03 as (  select  documento,  count (documento) as flagPlacas  from cte_baseHabilitada_03Porto  group by documento  having flagPlacas > 1  order by flagPlacas desc  ),    cte_baseHabilitada_03PFCom1Porto as (  select  a.documento::float,  a.anomes ::int,  a.agregado,  a.anomescancelamento::int  from cte_baseHabilitada_03Porto a  left join cte_flagPlacasasPorto_03 b on a.documento = b.documento  where b.flagPlacas is null and len(a.documento) <=11  ),    cte_BasePF_Clientes1Placa_03 as (  select * from cte_baseHabilitada_03PFCom1PlacaSemPorto  union  select * from cte_baseHabilitada_03PFCom1Porto  )    select * from cte_BasePF_Clientes1Placa_03 "

In [30]:
cte_BasePF_Clientes1Placa_02 = "WITH cte_MesRef_02 AS (  select to_char(date_trunc('month', current_date - interval '2 month'), 'YYYYMM') as MesRefVar  ),    cte_baseHabilitada_02 as (  select distinct  d.documento*1 :: float as documento,  date_part(year, a.DataHabilitada) ||lpad(date_part(month, a.DataHabilitada),2,'0') as anomes,  a.agregado as agregado,  date_part(year, b.datacancelamento) ||lpad(date_part(month, b.datacancelamento),2,'0') as anoMesCancelamento  from cntcar_work.vw_redshift_ciclodevida_basehabilitada_stage as a  left join cntcar_work.tb_conectcar_dbo_adesao_stage as b on a.adesaoid = b.adesaoid  left join cntcar_work.tb_conectcar_dbo_veiculo_stage as c on b.veiculoID = c.veiculo_id  left join conectcar.cntcar_work.tb_conectcar_dbo_cliente_stage as d on b.Clienteid = d.Clienteid  where anomes = (SELECT MesRefVar FROM cte_MesRef_02)  and len(documento) <= 11  ),    cte_flagPlacasas_02 as (  select  documento,  count (documento) as flagPlacas  from cte_baseHabilitada_02  group by documento  having flagPlacas > 1  order by flagPlacas desc  ),    cte_baseHabilitada_02PFCom1PlacaSemPorto as (  select  a.documento ::float,  a.anomes ::int,  a.agregado,  a.anomescancelamento::int  from cte_baseHabilitada_02 a  left join cte_flagPlacasas_02 b  on a.documento = b.documento  where b.flagPlacas is NULL  ),    cte_baseHabilitada_02Porto as (  select distinct  regexp_replace(c.codigointernoparceiro,'[^0-9]+', '') as Documento,  date_part(year, bh.datahabilitada) ||lpad(date_part(month, bh.datahabilitada),2,'0') as anomes,  p.agregado,  date_part(year, a.datacancelamento) ||lpad(date_part(month, a.datacancelamento),2,'0') as anomescancelamento  from cntcar_work.vw_redshift_ciclodevida_basehabilitada_stage bh  left outer join cntcar_work.tb_parceiro_dbo_ativacaocontrato_stage ac on ac.adesaoid = bh.adesaoid  left outer join cntcar_work.tb_parceiro_dbo_contrato_stage c on c.contratoid = ac.contratoid  left outer join cntcar_work.vw_redshift_ciclodevida_ativacao a on a.adesaoid = ac.adesaoid  left outer join cntcar_work.vw_ciclodevida_portifolio p on p.adesaoid = bh.adesaoid  where  upper(p.agregado) = 'PORTO SEGURO'  and upper(p.detalhado) not in ('CARROFACIL','CARRO FACIL','CARRO FÁCIL')  and upper(a.nomeplano) <> 'LOCADORA'  and lpad(date_part(year, cast(bh.datahabilitada as date)), 4, '0') ||lpad(date_part(month, cast(bh.datahabilitada as date)),2,'0') = (SELECT MesRefVar FROM cte_MesRef_02)  order by  1, 2  ),    cte_flagPlacasasPorto_02 as (  select  documento,  count (documento) as flagPlacas  from cte_baseHabilitada_02Porto  group by documento  having flagPlacas > 1  order by flagPlacas desc  ),    cte_baseHabilitada_02PFCom1Porto as (  select  a.documento::float,  a.anomes ::int,  a.agregado,  a.anomescancelamento::int  from cte_baseHabilitada_02Porto a  left join cte_flagPlacasasPorto_02 b on a.documento = b.documento  where b.flagPlacas is null and len(a.documento) <=11  ),    cte_BasePF_Clientes1Placa_02 as (  select * from cte_baseHabilitada_02PFCom1PlacaSemPorto  union  select * from cte_baseHabilitada_02PFCom1Porto  )    select * from cte_BasePF_Clientes1Placa_02 "

In [31]:
cte_BasePF_Clientes1Placa_01 = "WITH cte_MesRef_01 AS (  select to_char(date_trunc('month', current_date - interval '1 month'), 'YYYYMM') as MesRefVar  ),    cte_baseHabilitada_01 as (  select distinct  d.documento*1 :: float as documento,  date_part(year, a.DataHabilitada) ||lpad(date_part(month, a.DataHabilitada),2,'0') as anomes,  a.agregado as agregado,  date_part(year, b.datacancelamento) ||lpad(date_part(month, b.datacancelamento),2,'0') as anoMesCancelamento  from cntcar_work.vw_redshift_ciclodevida_basehabilitada_stage as a  left join cntcar_work.tb_conectcar_dbo_adesao_stage as b on a.adesaoid = b.adesaoid  left join cntcar_work.tb_conectcar_dbo_veiculo_stage as c on b.veiculoID = c.veiculo_id  left join conectcar.cntcar_work.tb_conectcar_dbo_cliente_stage as d on b.Clienteid = d.Clienteid  where anomes = (SELECT MesRefVar FROM cte_MesRef_01)  and len(documento) <= 11  ),    cte_flagPlacasas_01 as (  select  documento,  count (documento) as flagPlacas  from cte_baseHabilitada_01  group by documento  having flagPlacas > 1  order by flagPlacas desc  ),    cte_baseHabilitada_01PFCom1PlacaSemPorto as (  select  a.documento ::float,  a.anomes ::int,  a.agregado,  a.anomescancelamento::int  from cte_baseHabilitada_01 a  left join cte_flagPlacasas_01 b  on a.documento = b.documento  where b.flagPlacas is NULL  ),    cte_baseHabilitada_01Porto as (  select distinct  regexp_replace(c.codigointernoparceiro,'[^0-9]+', '') as Documento,  date_part(year, bh.datahabilitada) ||lpad(date_part(month, bh.datahabilitada),2,'0') as anomes,  p.agregado,  date_part(year, a.datacancelamento) ||lpad(date_part(month, a.datacancelamento),2,'0') as anomescancelamento  from cntcar_work.vw_redshift_ciclodevida_basehabilitada_stage bh  left outer join cntcar_work.tb_parceiro_dbo_ativacaocontrato_stage ac on ac.adesaoid = bh.adesaoid  left outer join cntcar_work.tb_parceiro_dbo_contrato_stage c on c.contratoid = ac.contratoid  left outer join cntcar_work.vw_redshift_ciclodevida_ativacao a on a.adesaoid = ac.adesaoid  left outer join cntcar_work.vw_ciclodevida_portifolio p on p.adesaoid = bh.adesaoid  where  upper(p.agregado) = 'PORTO SEGURO'  and upper(p.detalhado) not in ('CARROFACIL','CARRO FACIL','CARRO FÁCIL')  and upper(a.nomeplano) <> 'LOCADORA'  and lpad(date_part(year, cast(bh.datahabilitada as date)), 4, '0') ||lpad(date_part(month, cast(bh.datahabilitada as date)),2,'0') = (SELECT MesRefVar FROM cte_MesRef_01)  order by  1, 2  ),    cte_flagPlacasasPorto_01 as (  select  documento,  count (documento) as flagPlacas  from cte_baseHabilitada_01Porto  group by documento  having flagPlacas > 1  order by flagPlacas desc  ),    cte_baseHabilitada_01PFCom1Porto as (  select  a.documento::float,  a.anomes ::int,  a.agregado,  a.anomescancelamento::int  from cte_baseHabilitada_01Porto a  left join cte_flagPlacasasPorto_01 b on a.documento = b.documento  where b.flagPlacas is null and len(a.documento) <=11  ),    cte_BasePF_Clientes1Placa_01 as (  select * from cte_baseHabilitada_01PFCom1PlacaSemPorto  union  select * from cte_baseHabilitada_01PFCom1Porto  )    select * from cte_BasePF_Clientes1Placa_01 "

In [32]:
cte_BasePF_Clientes1Placa_00 = "WITH cte_MesRef_00 AS (  select to_char(date_trunc('month', current_date), 'YYYYMM') as MesRefVar  ),    cte_baseHabilitada_00 as (  select distinct  d.documento*1 :: float as documento,  date_part(year, a.DataHabilitada) ||lpad(date_part(month, a.DataHabilitada),2,'0') as anomes,  a.agregado as agregado,  date_part(year, b.datacancelamento) ||lpad(date_part(month, b.datacancelamento),2,'0') as anoMesCancelamento  from cntcar_work.vw_redshift_ciclodevida_basehabilitada_stage as a  left join cntcar_work.tb_conectcar_dbo_adesao_stage as b on a.adesaoid = b.adesaoid  left join cntcar_work.tb_conectcar_dbo_veiculo_stage as c on b.veiculoID = c.veiculo_id  left join conectcar.cntcar_work.tb_conectcar_dbo_cliente_stage as d on b.Clienteid = d.Clienteid  where anomes = (SELECT MesRefVar FROM cte_MesRef_00)  and len(documento) <= 11  ),    cte_flagPlacasas_00 as (  select  documento,  count (documento) as flagPlacas  from cte_baseHabilitada_00  group by documento  having flagPlacas > 1  order by flagPlacas desc  ),    cte_baseHabilitada_00PFCom1PlacaSemPorto as (  select  a.documento ::float,  a.anomes ::int,  a.agregado,  a.anomescancelamento::int  from cte_baseHabilitada_00 a  left join cte_flagPlacasas_00 b  on a.documento = b.documento  where b.flagPlacas is NULL  ),    cte_baseHabilitada_00Porto as (  select distinct  regexp_replace(c.codigointernoparceiro,'[^0-9]+', '') as Documento,  date_part(year, bh.datahabilitada) ||lpad(date_part(month, bh.datahabilitada),2,'0') as anomes,  p.agregado,  date_part(year, a.datacancelamento) ||lpad(date_part(month, a.datacancelamento),2,'0') as anomescancelamento  from cntcar_work.vw_redshift_ciclodevida_basehabilitada_stage bh  left outer join cntcar_work.tb_parceiro_dbo_ativacaocontrato_stage ac on ac.adesaoid = bh.adesaoid  left outer join cntcar_work.tb_parceiro_dbo_contrato_stage c on c.contratoid = ac.contratoid  left outer join cntcar_work.vw_redshift_ciclodevida_ativacao a on a.adesaoid = ac.adesaoid  left outer join cntcar_work.vw_ciclodevida_portifolio p on p.adesaoid = bh.adesaoid  where  upper(p.agregado) = 'PORTO SEGURO'  and upper(p.detalhado) not in ('CARROFACIL','CARRO FACIL','CARRO FÁCIL')  and upper(a.nomeplano) <> 'LOCADORA'  and lpad(date_part(year, cast(bh.datahabilitada as date)), 4, '0') ||lpad(date_part(month, cast(bh.datahabilitada as date)),2,'0') = (SELECT MesRefVar FROM cte_MesRef_00)  order by  1, 2  ),    cte_flagPlacasasPorto_00 as (  select  documento,  count (documento) as flagPlacas  from cte_baseHabilitada_00Porto  group by documento  having flagPlacas > 1  order by flagPlacas desc  ),    cte_baseHabilitada_00PFCom1Porto as (  select  a.documento::float,  a.anomes ::int,  a.agregado,  a.anomescancelamento::int  from cte_baseHabilitada_00Porto a  left join cte_flagPlacasasPorto_00 b on a.documento = b.documento  where b.flagPlacas is null and len(a.documento) <=11  ),    cte_BasePF_Clientes1Placa_00 as (  select * from cte_baseHabilitada_00PFCom1PlacaSemPorto  union  select * from cte_baseHabilitada_00PFCom1Porto  )    select * from cte_BasePF_Clientes1Placa_00 "

**Cruzando os meses possuindo como referencia o primeiro mes**

In [33]:
consolidando_meses = "with consolidando_meses as(  select distinct  a.documento,  a.anomes,  a.anomescancelamento,  a.agregado as agregadoJun22,  b.agregado as agregadoJul22,  c.agregado as agregadoAgo22,  d.agregado as agregadoSet22,  e.agregado as agregadoOut22,  f.agregado as agregadoNov22,  g.agregado as agregadoDez22,  h.agregado as agregadoJan23,  i.agregado as agregadoFev23,  j.agregado as agregadoMar23  from cte_BasePF_Clientes1Placa_09 	   as a         left join cte_BasePF_Clientes1Placa_08 as b  on a.documento = b.documento  left join cte_BasePF_Clientes1Placa_07 as c  on a.documento = c.documento  left join cte_BasePF_Clientes1Placa_06 as d  on a.documento = d.documento  left join cte_BasePF_Clientes1Placa_05 as e  on a.documento = e.documento  left join cte_BasePF_Clientes1Placa_04 as f  on a.documento = f.documento  left join cte_BasePF_Clientes1Placa_03 as g  on a.documento = g.documento  left join cte_BasePF_Clientes1Placa_02 as h  on a.documento = h.documento  left join cte_BasePF_Clientes1Placa_01 as i  on a.documento = i.documento  left join cte_BasePF_Clientes1Placa_00 as j  on a.documento = j.documento  ),  select   agregadoJun22,  agregadoJul22,  agregadoAgo22,  agregadoSet22,  agregadoOut22,  agregadoNov22,  agregadoDez22,  agregadoJan23,  agregadoFev23,  agregadoMar23  count(DISTINCT documento) DocDistinct,  count(documento) DocDup  from consolidando_meses  group by  agregadoJun22,  agregadoJul22,  agregadoAgo22,  agregadoSet22,  agregadoOut22,  agregadoNov22,  agregadoDez22,  agregadoJan23,  agregadoFev23,  agregadoMar23  order by DocDistinct desc    "

In [ ]:
cte_BasePF_Clientes1Placa_09 = sqlio.read_sql_query(cte_BasePF_Clientes1Placa_09, conn) 
cte_BasePF_Clientes1Placa_08 = sqlio.read_sql_query(cte_BasePF_Clientes1Placa_08, conn) 
cte_BasePF_Clientes1Placa_07 = sqlio.read_sql_query(cte_BasePF_Clientes1Placa_07, conn) 
cte_BasePF_Clientes1Placa_06 = sqlio.read_sql_query(cte_BasePF_Clientes1Placa_06, conn) 
cte_BasePF_Clientes1Placa_05 = sqlio.read_sql_query(cte_BasePF_Clientes1Placa_05, conn) 
cte_BasePF_Clientes1Placa_04 = sqlio.read_sql_query(cte_BasePF_Clientes1Placa_04, conn) 
cte_BasePF_Clientes1Placa_03 = sqlio.read_sql_query(cte_BasePF_Clientes1Placa_03, conn) 
cte_BasePF_Clientes1Placa_02 = sqlio.read_sql_query(cte_BasePF_Clientes1Placa_02, conn)
cte_BasePF_Clientes1Placa_01 = sqlio.read_sql_query(cte_BasePF_Clientes1Placa_01, conn)  
cte_BasePF_Clientes1Placa_00 = sqlio.read_sql_query(cte_BasePF_Clientes1Placa_00, conn)  

ConsolidaMeses = sqlio.read_sql_query(sql, conn) 

In [6]:
ConsolidaMeses = sqlio.read_sql_query(sql, conn) 

c:\Users\raul.correa\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


DatabaseError: Execution failed on sql 'WITH  cte_MesRef AS (      SELECT                              to_char(date_trunc('month', current_date - interval '7 month'), 'YYYYMM') as MesRefVar ),                   cte_baseHabilitada as (       	select distinct      	d.documento*1 :: float as documento,      	date_part(year, a.DataHabilitada) ||lpad(date_part(month, a.DataHabilitada),2,'0') as anomes,      	a.agregado as agregado,      	date_part(year, b.datacancelamento) ||lpad(date_part(month, b.datacancelamento),2,'0') as anoMesCancelamento      	from cntcar_work.vw_redshift_ciclodevida_basehabilitada_stage as a      	left join cntcar_work.tb_conectcar_dbo_adesao_stage as b on a.adesaoid = b.adesaoid                        left join cntcar_work.tb_conectcar_dbo_veiculo_stage as c on b.veiculoID = c.veiculo_id      	left join conectcar.cntcar_work.tb_conectcar_dbo_cliente_stage as d on b.Clienteid = d.Clienteid      	      	where anomes = (SELECT MesRefVar FROM cte_MesRef)       	and len